<a href="https://colab.research.google.com/github/Juunary/bicycle_website/blob/main/page_bicycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#사이트 기재

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*lightgbm.*')

#데이터 값 받기
Get_year = int(input())
Get_month = int(input())
Get_date = int(input())
Get_selectedCity=0
Get_selectedModel=2

data = "2023-10-27 서울 랜덤포레스트 서울(108) 2023-10-27 15.7 NULL 1.9 71.1 10.07 NULL 16.5"

# 사용자 입력 데이터 처리 함수
def process_user_data(data):
    data_parts = data.split()
    year, mnth, day = map(int, data_parts[0].split('-'))
    values = [float(val) if val != "NULL" else np.nan for val in data_parts[5:12]]
    avg_tmp, day_p, avg_wind, avg_rhum, t_sd, snow, avg_gtmp = values

    if mnth in [12, 1, 2]:
        season = 3
    elif mnth in [3, 4, 5]:
        season = 0
    elif mnth in [6, 7, 8]:
        season = 1
    elif mnth in [9, 10, 11]:
        season = 2
    result=pd.DataFrame([{'year': year, 'mnth': mnth, 'day': day, 'avg_tmp': avg_tmp, 'day_p': day_p,
                          'avg_wind': avg_wind, 'avg_rhum': avg_rhum, 't_sd': t_sd, 'snow': snow,
                          'avg_gtmp': avg_gtmp, 'season': season}])
    result = result.fillna(result.mean())
    return result

def process_and_encode_data(data, df):
    # 사용자 데이터 처리
    result_df = process_user_data(data)

    # 연도, 월, 일 컬럼 추출
    result_df['year'] = Get_year
    result_df['mnth'] = Get_month
    result_df['day'] = Get_date

    # 원-핫 인코딩 처리
    result_df = pd.get_dummies(result_df, columns=['day', 'year', 'mnth', 'season'])

    # 기존 데이터에 존재하는 모든 컬럼을 포함하도록 처리 (누락된 컬럼은 0으로 채움)
    for col in df.columns:
        if col not in result_df.columns:
            result_df[col] = 0

    # 'day_p'와 'snow' 컬럼의 null 값을 0으로 대체
    result_df['day_p'].fillna(0, inplace=True)
    result_df['snow'].fillna(0, inplace=True)

    # 훈련 데이터에 있는 모든 feature를 포함하도록 예측 데이터를 조정
    for col in df.drop(['CNT'], axis=1).columns:
        if col not in result_df.columns:
            result_df[col] = 0  # 없는 feature는 0으로 채웁니다.

    # 예측 데이터에 있는 모든 feature가 훈련 데이터에도 있는지 확인하고, 그렇지 않은 경우 해당 feature를 제거
    columns_to_remove = [col for col in result_df.columns if col not in df.drop(['CNT'], axis=1).columns]
    result_df.drop(columns=columns_to_remove, inplace=True)

    return result_df

def load_and_process_data():
    # 데이터 로딩
    df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

    # 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['mnth'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    # 필요한 컬럼 선택
    selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
    df = df[selected_columns]

    # 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
    df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

    # 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
    df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

    # 결측치 처리 (평균값으로 대체)
    df = df.fillna(df.mean())

    # X와 y 데이터 설정
    X = df.drop(['CNT'], axis=1).values.astype(np.float32)
    y = df['CNT'].values.astype(np.float32)

    return X, y, df

# 모델 훈련 및 예측 함수
def train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    result_df = process_and_encode_data(data, df)
    predicted_CNT = model.predict(result_df.values.astype(np.float32))
    predicted_CNT_rounded = round(predicted_CNT[0])
    print(f"{model_name} predicted_cnt(rounded): {predicted_CNT_rounded}")

# 모델별 훈련 및 예측 함수 정의
def train_and_predict_lgbm(X_train, X_test, y_train, y_test, data, df):
    model = lgb.LGBMRegressor(max_depth=30, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "LightGBM")

def train_and_predict_xgboost(X_train, X_test, y_train, y_test, data, df):
    model = xgb.XGBRegressor(learning_rate=0.2, max_depth=3, n_estimators=100, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "XGBoost")

def train_and_predict_rf(X_train, X_test, y_train, y_test, data, df):
    model = RandomForestRegressor(n_estimators=300, max_features='sqrt', max_depth=20, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Random Forest")

def train_and_predict_gb(X_train, X_test, y_train, y_test, data, df):
    model = GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Gradient Boosting")

def train_and_predict_dt(X_train, X_test, y_train, y_test, data, df):
    model = DecisionTreeRegressor(max_depth=20, min_samples_split=10, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Decision Tree")

def train_and_predict_lr(X_train, X_test, y_train, y_test, data, df):
    model = LinearRegression()
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Linear Regression")

# 데이터 로딩 및 처리
X, y, df = load_and_process_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 선택에 따라 해당 모델 훈련 및 예측
if Get_selectedModel == 1:
    train_and_predict_lgbm(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 2:
    train_and_predict_xgboost(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 3:
    train_and_predict_rf(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 4:
    train_and_predict_gb(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 5:
    train_and_predict_dt(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 6:
    train_and_predict_lr(X_train, X_test, y_train, y_test, data, df)

else:
    print("error_wrong_number")

2023
11
14
XGBoost predicted_cnt(rounded): 90079


#종합 파일 연습용

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

#데이터 값 받기
Get_year = int(input())
Get_month = int(input())
Get_date = int(input())
Get_selectedCity=0
Get_selectedModel=2

data = "2023-10-27 서울 랜덤포레스트 서울(108) 2023-10-27 15.7 NULL 1.9 71.1 10.07 NULL 16.5"

# 사용자 입력 데이터 처리 함수
def process_user_data(data):
    data_parts = data.split()
    year, mnth, day = map(int, data_parts[0].split('-'))
    values = [float(val) if val != "NULL" else np.nan for val in data_parts[5:12]]
    avg_tmp, day_p, avg_wind, avg_rhum, t_sd, snow, avg_gtmp = values

    if mnth in [12, 1, 2]:
        season = 3
    elif mnth in [3, 4, 5]:
        season = 0
    elif mnth in [6, 7, 8]:
        season = 1
    elif mnth in [9, 10, 11]:
        season = 2
    result=pd.DataFrame([{'year': year, 'mnth': mnth, 'day': day, 'avg_tmp': avg_tmp, 'day_p': day_p,
                          'avg_wind': avg_wind, 'avg_rhum': avg_rhum, 't_sd': t_sd, 'snow': snow,
                          'avg_gtmp': avg_gtmp, 'season': season}])
    result = result.fillna(result.mean())
    return result

def process_and_encode_data(data, df):
    # 사용자 데이터 처리
    result_df = process_user_data(data)

    # 연도, 월, 일 컬럼 추출
    result_df['year'] = Get_year
    result_df['mnth'] = Get_month
    result_df['day'] = Get_date

    # 원-핫 인코딩 처리
    result_df = pd.get_dummies(result_df, columns=['day', 'year', 'mnth', 'season'])

    # 기존 데이터에 존재하는 모든 컬럼을 포함하도록 처리 (누락된 컬럼은 0으로 채움)
    for col in df.columns:
        if col not in result_df.columns:
            result_df[col] = 0

    # 'day_p'와 'snow' 컬럼의 null 값을 0으로 대체
    result_df['day_p'].fillna(0, inplace=True)
    result_df['snow'].fillna(0, inplace=True)

    # 훈련 데이터에 있는 모든 feature를 포함하도록 예측 데이터를 조정
    for col in df.drop(['CNT'], axis=1).columns:
        if col not in result_df.columns:
            result_df[col] = 0  # 없는 feature는 0으로 채웁니다.

    # 예측 데이터에 있는 모든 feature가 훈련 데이터에도 있는지 확인하고, 그렇지 않은 경우 해당 feature를 제거
    columns_to_remove = [col for col in result_df.columns if col not in df.drop(['CNT'], axis=1).columns]
    result_df.drop(columns=columns_to_remove, inplace=True)

    return result_df

def load_and_process_data():
    # 데이터 로딩
    df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

    # 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['mnth'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    # 필요한 컬럼 선택
    selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
    df = df[selected_columns]

    # 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
    df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

    # 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
    df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

    # 결측치 처리 (평균값으로 대체)
    df = df.fillna(df.mean())

    # X와 y 데이터 설정
    X = df.drop(['CNT'], axis=1).values.astype(np.float32)
    y = df['CNT'].values.astype(np.float32)

    return X, y, df

# 모델 훈련 및 예측 함수
def train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    result_df = process_and_encode_data(data, df)
    predicted_CNT = model.predict(result_df.values.astype(np.float32))
    predicted_CNT_rounded = round(predicted_CNT[0])
    print(f"{model_name} 모델에 의한 예상 CNT (반올림): {predicted_CNT_rounded}")

# 모델별 훈련 및 예측 함수 정의
def train_and_predict_lgbm(X_train, X_test, y_train, y_test, data, df):
    model = lgb.LGBMRegressor(max_depth=30, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "LightGBM")

def train_and_predict_xgboost(X_train, X_test, y_train, y_test, data, df):
    model = xgb.XGBRegressor(learning_rate=0.2, max_depth=3, n_estimators=100, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "XGBoost")

def train_and_predict_rf(X_train, X_test, y_train, y_test, data, df):
    model = RandomForestRegressor(n_estimators=300, max_features='sqrt', max_depth=20, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Random Forest")

def train_and_predict_gb(X_train, X_test, y_train, y_test, data, df):
    model = GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Gradient Boosting")

def train_and_predict_dt(X_train, X_test, y_train, y_test, data, df):
    model = DecisionTreeRegressor(max_depth=20, min_samples_split=10, random_state=42)
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Decision Tree")

def train_and_predict_lr(X_train, X_test, y_train, y_test, data, df):
    model = LinearRegression()
    train_and_predict_model(model, X_train, X_test, y_train, y_test, data, df, "Linear Regression")

# 데이터 로딩 및 처리
X, y, df = load_and_process_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 선택에 따라 해당 모델 훈련 및 예측
if Get_selectedModel == 1:
    train_and_predict_lgbm(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 2:
    train_and_predict_xgboost(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 3:
    train_and_predict_rf(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 4:
    train_and_predict_gb(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 5:
    train_and_predict_dt(X_train, X_test, y_train, y_test, data, df)

elif Get_selectedModel == 6:
    train_and_predict_lr(X_train, X_test, y_train, y_test, data, df)

else:
    print("잘못된 모델 선택입니다. 1부터 6까지의 숫자 중 하나를 선택해주세요.")

2023
10
26
XGBoost 모델에 의한 예상 CNT (반올림): 90079


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
models = {
    '랜덤 포레스트': RandomForestRegressor(max_depth=30, n_estimators=200, random_state=42),
    '그래디언트 부스팅': GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42),
    '의사결정 트리': DecisionTreeRegressor(max_depth=20, min_samples_split=10, random_state=42),
    '선형 회귀': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(max_depth=30, random_state=42),
    'XGBoost': xgb.XGBRegressor(learning_rate=0.2, max_depth=3, n_estimators=100, random_state=42),
}

results = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)
    results.append({'모델': model_name, 'RMSE': rmse, 'MAE': mae, 'R-squared': r_squared})

# 결과 정렬
results_sorted = sorted(results, key=lambda x: x['RMSE'])

# 결과 출력
for result in results_sorted:
    print(f"{result['모델']} 모델:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"MAE: {result['MAE']:.2f}")
    print(f"R-squared: {result['R-squared']:.2f}")
    print("\n")

#모델 모음

##모델-xgboost, 결과값 출력

In [ ]:
#xgboost
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_error
import numpy as np

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 훈련
xg_model = xgb.XGBRegressor()  # 이 부분을 수정했습니다.
xg_model.fit(X_train, y_train)

# 훈련된 XGBoost 모델로 예측 수행
xg_pred = xg_model.predict(X_test)

# 사용자 입력 데이터 처리 함수
def process_user_data(data):
    data_parts = data.split()
    year, mnth, day = map(int, data_parts[0].split('-'))
    values = [float(val) if val != "NULL" else np.nan for val in data_parts[5:12]]
    avg_tmp, day_p, avg_wind, avg_rhum, t_sd, snow, avg_gtmp = values

    if mnth in [12, 1, 2]:
        season = 3
    elif mnth in [3, 4, 5]:
        season = 0
    elif mnth in [6, 7, 8]:
        season = 1
    elif mnth in [9, 10, 11]:
        season = 2

    return pd.DataFrame([{'year': year, 'mnth': mnth, 'day': day, 'avg_tmp': avg_tmp, 'day_p': day_p,
                          'avg_wind': avg_wind, 'avg_rhum': avg_rhum, 't_sd': t_sd, 'snow': snow,
                          'avg_gtmp': avg_gtmp, 'season': season}])

data = "2023-10-27 서울 랜덤포레스트 서울(108) 2023-10-27 15.7 NULL 1.9 71.1 10.07 NULL 16.5"
result_df = process_user_data(data)

# 원-핫 인코딩 처리
result_df = pd.get_dummies(result_df, columns=['day', 'year', 'mnth', 'season'])

# 기존 데이터에 존재하는 모든 컬럼을 포함하도록 처리 (누락된 컬럼은 0으로 채움)
for col in df.columns:
    if col not in result_df.columns:
        result_df[col] = 0

# 컬럼 순서를 맞춤
result_df = result_df[df.drop(['CNT'], axis=1).columns]

# 결측치 처리 (평균값으로 대체)
result_df = result_df.fillna(df.mean())

# 사용자 데이터에 대한 예측 수행
user_pred = xg_model.predict(result_df.values.astype(np.float32))

# 예측값 반올림 후 출력
print(f"Rounded Predicted Count: {round(user_pred[0])}")

Rounded Predicted Count: 153115


In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

#데이터 값 받기
Get_year = 2022
Get_month = 10
Get_date = 26
Get_selectedCity=0
Get_selectedModel=1

# 사용자 입력 데이터 처리 함수
def process_user_data(data):
    data_parts = data.split()
    year, mnth, day = map(int, data_parts[0].split('-'))
    values = [float(val) if val != "NULL" else np.nan for val in data_parts[5:12]]
    avg_tmp, day_p, avg_wind, avg_rhum, t_sd, snow, avg_gtmp = values

    if mnth in [12, 1, 2]:
        season = 3
    elif mnth in [3, 4, 5]:
        season = 0
    elif mnth in [6, 7, 8]:
        season = 1
    elif mnth in [9, 10, 11]:
        season = 2

    return pd.DataFrame([{'year': year, 'mnth': mnth, 'day': day, 'avg_tmp': avg_tmp, 'day_p': day_p,
                          'avg_wind': avg_wind, 'avg_rhum': avg_rhum, 't_sd': t_sd, 'snow': snow,
                          'avg_gtmp': avg_gtmp, 'season': season}])

data = "2023-10-27 서울 랜덤포레스트 서울(108) 2023-10-27 15.7 NULL 1.9 71.1 10.07 NULL 16.5"
result_df = process_user_data(data)

result_df['year'] = Get_year
result_df['mnth'] = Get_month
result_df['day'] = Get_date

# 원-핫 인코딩 처리
result_df = pd.get_dummies(result_df, columns=['day', 'year', 'mnth', 'season'])

# 기존 데이터에 존재하는 모든 컬럼을 포함하도록 처리 (누락된 컬럼은 0으로 채움)
for col in df.columns:
    if col not in result_df.columns:
        result_df[col] = 0

# 컬럼 순서를 맞춤
result_df = result_df[df.drop(['CNT'], axis=1).columns]

# 결측치 처리 (평균값으로 대체)
result_df = result_df.fillna(df.mean())


# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 최적의 하이퍼파라미터로 랜덤 포레스트 모델 생성 및 훈련
best_rf_model = RandomForestRegressor(n_estimators=300, max_features='sqrt', max_depth=20, random_state=42)
best_rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = best_rf_model.predict(X_test)

# 예측 수행
predicted_CNT = best_rf_model.predict(result_df.values.astype(np.float32))

# 예측 결과를 반올림
predicted_CNT_rounded = round(predicted_CNT[0])

# 예측 결과 출력
print(f"Predicted CNT (Rounded): {predicted_CNT_rounded}")

# XGBoost 모델 훈련
xg_model = xgb.XGBRegressor()  # 이 부분을 수정했습니다.
xg_model.fit(X_train, y_train)

# 훈련된 XGBoost 모델로 예측 수행
xg_pred = xg_model.predict(X_test)

# 사용자 데이터에 대한 예측 수행
user_pred = xg_model.predict(result_df.values.astype(np.float32))

# 예측값 반올림 후 출력
print(f"Rounded Predicted Count: {round(user_pred[0])}")

Predicted CNT (Rounded): 138320
Rounded Predicted Count: 157140


##여러 모델 돌려봐서 종류 선택하기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
import tensorflow as tf
from tensorflow import keras
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
models = {
    '랜덤 포레스트': RandomForestRegressor(random_state=42),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    '의사결정 트리': DecisionTreeRegressor(random_state=42),
    '선형 회귀': LinearRegression(),
    '그래디언트 부스팅': GradientBoostingRegressor(random_state=42),
    '서포트 벡터 머신': SVR(),
    '나이브 베이즈': GaussianNB(),
    'MLP 모델': keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
    ]),
    'RNN 모델': keras.Sequential([
        keras.layers.LSTM(64, input_shape=(X_train.shape[1], 1), activation='relu'),
        keras.layers.Dense(1)
    ])
}

# 각 모델의 예측값 구하기 및 평가
results = []
for model_name, model in models.items():
    if '모델' in model_name:  # 모델 이름에 '모델'이 포함된 경우만
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    else:
        model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)
    results.append({'모델': model_name, 'RMSE': rmse, 'MAE': mae, 'R-squared': r_squared})

# 결과 정렬
results_sorted = sorted(results, key=lambda x: x['RMSE'])

# 결과 출력
for result in results_sorted:
    print(f"{result['모델']} 모델:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"MAE: {result['MAE']:.2f}")
    print(f"R-squared: {result['R-squared']:.2f}")
    print("\n")

<ipython-input-8-a4c640d7d993>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CNT'] = df['CNT'].str.replace(',', '').astype(float)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000381 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1270
[LightGBM] [Info] Number of data points in the train set: 1604, number of used features: 60
[LightGBM] [Info] Start training from score 73745.144015
13/13 [==============================] - 0s 12ms/step
LightGBM 모델:
RMSE: 14028.69
MAE: 8739.13
R-squared: 0.92


랜덤 포레스트 모델:
RMSE: 14558.71
MAE: 9299.78
R-squared: 0.91


그래디언트 부스팅 모델:
RMSE: 14760.52
MAE: 9625.47
R-squared: 0.91


의사결정 트리 모델:
RMSE: 17496.67
MAE: 11544.72
R-squared: 0.88


선형 회귀 모델:
RMSE: 21190.47
MAE: 15568.02
R-squared: 0.82


MLP 모델 모델:
RMSE: 38763.39
MAE: 30882.75
R-squared: 0.39


나이브 베이즈 모델:
RMSE: 47178.27
MAE: 36149.20
R-squared: 0.10


서포트 벡터 머신 모델:
RMSE: 49850.09
MAE: 41107.87
R-squared: -0.01


RNN 모델 모델:
RMSE: 59308.35
MAE: 45369.29
R-squared: -0.43




In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
models = {
    '랜덤 포레스트': RandomForestRegressor(random_state=42),
    '그래디언트 부스팅': GradientBoostingRegressor(random_state=42),
    '의사결정 트리': DecisionTreeRegressor(random_state=42),
    '선형 회귀': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    '서포트 벡터 머신': SVR(),
    '나이브 베이즈': GaussianNB(),
    'MLP 모델': keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ]),
    '두 번째 신경망 모델': keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
}

# 각 모델의 예측값 구하기 및 평가
results = []
for model_name, model in models.items():
    if '모델' in model_name:  # 모델 이름에 '모델'이 포함된 경우만
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    else:
        model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)
    results.append({'모델': model_name, 'RMSE': rmse, 'MAE': mae, 'R-squared': r_squared})

# 결과 정렬
results_sorted = sorted(results, key=lambda x: x['RMSE'])

# 결과 출력
for result in results_sorted:
    print(f"{result['모델']} 모델:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"MAE: {result['MAE']:.2f}")
    print(f"R-squared: {result['R-squared']:.2f}")
    print("\n")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000345 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1270
[LightGBM] [Info] Number of data points in the train set: 1604, number of used features: 60
[LightGBM] [Info] Start training from score 73745.144015
13/13 [==============================] - 0s 2ms/step
LightGBM 모델:
RMSE: 14028.69
MAE: 8739.13
R-squared: 0.92


XGBoost 모델:
RMSE: 14087.92
MAE: 9023.32
R-squared: 0.92


랜덤 포레스트 모델:
RMSE: 14558.71
MAE: 9299.78
R-squared: 0.91


그래디언트 부스팅 모델:
RMSE: 14760.52
MAE: 9625.47
R-squared: 0.91


의사결정 트리 모델:
RMSE: 17496.67
MAE: 11544.72
R-squared: 0.88


선형 회귀 모델:
RMSE: 21190.47
MAE: 15568.02
R-squared: 0.82


MLP 모델 모델:
RMSE: 34823.39
MAE: 27349.01
R-squared: 0.51


두 번째 신경망 모델 모델:
RMSE: 35618.15
MAE: 28017.73
R-squared: 0.48


나이브 베이즈 모델:
RMSE: 47178.27
MAE: 36149.20
R-squared: 0.10


서포트 벡터 머신 모델:
RMSE: 49850.09
MAE: 41107.87
R-squared: -0.01




In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 Label Encoding 처리 (season 컬럼에만 적용)
label_encoder = LabelEncoder()
df['season'] = label_encoder.fit_transform(df['season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Keras 모델 생성 함수
def create_keras_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 모델 생성
models = {
    '랜덤 포레스트': RandomForestRegressor(random_state=42),
    '그래디언트 부스팅': GradientBoostingRegressor(random_state=42),
    '의사결정 트리': DecisionTreeRegressor(random_state=42),
    '선형 회귀': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42),
    'MLP 모델': KerasRegressor(build_fn=create_keras_model, epochs=50, batch_size=32, verbose=0)
}

# 하이퍼파라미터 그리드 정의 (튜닝할 매개변수들)
param_grids = {
    '랜덤 포레스트': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30]
    },
    '그래디언트 부스팅': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    '의사결정 트리': {
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    '선형 회귀': {},
    'LightGBM': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'MLP 모델': {}
}

# 각 모델에 대한 하이퍼파라미터 튜닝
tuned_models = {}
for model_name, model in models.items():
    if model_name in param_grids:
        param_grid = param_grids[model_name]
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        tuned_models[model_name] = best_model
    else:
        tuned_models[model_name] = model

# 각 모델의 예측값 구하기 및 평가
results = []
for model_name, model in tuned_models.items():
    if '모델' in model_name:  # 모델 이름에 '모델'이 포함된 경우만
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    else:
        model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)
    results.append({'모델': model_name, 'RMSE': rmse, 'MAE': mae, 'R-squared': r_squared})

# 결과 정렬
results_sorted = sorted(results, key=lambda x: x['RMSE'])

# 결과 출력
for result in results_sorted:
    print(f"{result['모델']} 모델:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"MAE: {result['MAE']:.2f}")
    print(f"R-squared: {result['R-squared']:.2f}")
    print("\n")

NameError: ignored

##하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 모델 정의
rf_model = RandomForestRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search_rf.best_estimator_
print("최적의 랜덤 포레스트 모델:", best_rf_model)

최적의 랜덤 포레스트 모델: RandomForestRegressor(max_depth=30, n_estimators=200, random_state=42)


In [ ]:
# 그래디언트 부스팅 모델 정의
gb_model = GradientBoostingRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid_gb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search_gb = GridSearchCV(gb_model, param_grid_gb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_gb.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_gb_model = grid_search_gb.best_estimator_
print("최적의 그래디언트 부스팅 모델:", best_gb_model)

최적의 그래디언트 부스팅 모델: GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42)


In [ ]:
# 의사결정 트리 모델 정의
dt_model = DecisionTreeRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid_dt = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search_dt = GridSearchCV(dt_model, param_grid_dt, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_dt.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_dt_model = grid_search_dt.best_estimator_
print("최적의 의사결정 트리 모델:", best_dt_model)

최적의 의사결정 트리 모델: DecisionTreeRegressor(max_depth=20, min_samples_split=10, random_state=42)


In [ ]:
import lightgbm as lgb

# LightGBM 모델 정의
lgb_model = lgb.LGBMRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid_lgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 0.2]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search_lgb = GridSearchCV(lgb_model, param_grid_lgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lgb.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_lgb_model = grid_search_lgb.best_estimator_
print("최적의 LightGBM 모델:", best_lgb_model)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000078 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1221
[LightGBM] [Info] Number of data points in the train set: 1604, number of used features: 11
[LightGBM] [Info] Start training from score 73745.144015
최적의 LightGBM 모델: LGBMRegressor(max_depth=30, random_state=42)


In [ ]:
import xgboost as xgb

# XGBoost 모델 정의
xgb_model = xgb.XGBRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_xgb_model = grid_search_xgb.best_estimator_
print("최적의 XGBoost 모델:", best_xgb_model)

최적의 XGBoost 모델: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)


In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# 선형 회귀 모델 정의 (Ridge Regression)
lr_model = Ridge()

# 하이퍼파라미터 그리드 정의
param_grid_lr = {
    'alpha': [0.1, 1.0, 10.0]  # 여기서는 alpha를 조정해보겠습니다.
}

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝
grid_search_lr = GridSearchCV(lr_model, param_grid_lr, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_lr_model = grid_search_lr.best_estimator_
print("최적의 선형 회귀 모델 (Ridge Regression):", best_lr_model)

최적의 선형 회귀 모델 (Ridge Regression): Ridge()


#전체 모델 코드

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from tensorflow import keras
import tensorflow.keras as keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 각 모델에 대한 하이퍼파라미터 그리드 정의
param_grids = {
    '랜덤 포레스트': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30]
    },
    '그래디언트 부스팅': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    '의사결정 트리': {
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    '선형 회귀': {},
    'LightGBM': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'MLP 모델': {}
}

# 모델 이름과 최적 하이퍼파라미터 저장
best_models = {}

# 모델별로 하이퍼파라미터 튜닝과 평가 수행
for model_name, param_grid in param_grids.items():
    model = None
    if model_name == '랜덤 포레스트':
        model = RandomForestRegressor(random_state=42)
    elif model_name == '그래디언트 부스팅':
        model = GradientBoostingRegressor(random_state=42)
    elif model_name == '의사결정 트리':
        model = DecisionTreeRegressor(random_state=42)
    elif model_name == '선형 회귀':
        model = LinearRegression()
    elif model_name == 'LightGBM':
        model = lgb.LGBMRegressor(random_state=42)
    elif model_name == 'XGBoost':
        model = xgb.XGBRegressor(random_state=42)
    elif model_name == 'MLP 모델':
        def create_keras_model():
            model = keras.Sequential([
                keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
                keras.layers.Dropout(0.2),
                keras.layers.Dense(64, activation='relu'),
                keras.layers.Dense(1)
            ])
            model.compile(optimizer='adam', loss='mean_squared_error')
            return model
        model = KerasRegressor(build_fn=create_keras_model, epochs=50, batch_size=32, verbose=0)

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_

# 각 모델의 예측값 구하기 및 평가
results = []
for model_name, model in best_models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)
    results.append({'모델': model_name, 'RMSE': rmse, 'MAE': mae, 'R-squared': r_squared})

# 결과를 RMSE 기준으로 정렬
results_sorted = sorted(results, key=lambda x: x['RMSE'])

# 결과 출력
for result in results_sorted:
    print(f"최적의 {result['모델']} 모델:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"MAE: {result['MAE']:.2f}")
    print(f"R-squared: {result['R-squared']:.2f}")
    print("\n")

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Battle-of-the-Strongest-Statisticians/main/18-23%20Seoul_day.csv')

# 'date' 컬럼을 날짜 형식으로 변환하고 연, 월, 일 정보 추출
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['mnth'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# 필요한 컬럼 선택
selected_columns = ['day', 'year', 'mnth', 'avg_tmp', 'day_p', 'avg_wind', 'avg_rhum', 't_sd', 'snow', 'avg_gtmp', 'season', 'CNT']
df = df[selected_columns]

# 'CNT' 컬럼의 쉼표 제거하고 실수형으로 변환
df['CNT'] = df['CNT'].str.replace(',', '').astype(float)

# 범주형 데이터 원-핫 인코딩 처리 (필요한 컬럼에 대해서만 실행)
df = pd.get_dummies(df, columns=['day', 'year', 'mnth', 'season'])

# 결측치 처리 (평균값으로 대체)
df = df.fillna(df.mean())

# X와 y 데이터 설정
X = df.drop(['CNT'], axis=1).values.astype(np.float32)
y = df['CNT'].values.astype(np.float32)

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
models = {
    '랜덤 포레스트': RandomForestRegressor(max_depth=30, n_estimators=200, random_state=42),
    '그래디언트 부스팅': GradientBoostingRegressor(max_depth=4, n_estimators=200, random_state=42),
    '의사결정 트리': DecisionTreeRegressor(max_depth=20, min_samples_split=10, random_state=42),
    '선형 회귀': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(max_depth=30, random_state=42),
    'XGBoost': xgb.XGBRegressor(learning_rate=0.2, max_depth=3, n_estimators=100, random_state=42),
}

results = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r_squared = r2_score(y_test, predictions)
    results.append({'모델': model_name, 'RMSE': rmse, 'MAE': mae, 'R-squared': r_squared})

# 결과 정렬
results_sorted = sorted(results, key=lambda x: x['RMSE'])

# 결과 출력
for result in results_sorted:
    print(f"{result['모델']} 모델:")
    print(f"RMSE: {result['RMSE']:.2f}")
    print(f"MAE: {result['MAE']:.2f}")
    print(f"R-squared: {result['R-squared']:.2f}")
    print("\n")

<ipython-input-25-ef11856c07bd>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CNT'] = df['CNT'].str.replace(',', '').astype(float)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1270
[LightGBM] [Info] Number of data points in the train set: 1604, number of used features: 60
[LightGBM] [Info] Start training from score 73745.144015
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
그래디언트 부스팅 모델:
RMSE: 13606.55
MAE: 8823.61
R-squared: 0.92


LightGBM 모델:
RMSE: 14028.69
MAE: 8739.13
R-squared: 0.92


랜덤 포레스트 모델:
RMSE: 14460.43
MAE: 9301.26
R-squared: 0.91


XGBoost 모델:
RMSE: 14550.40
MAE: 9407.87
R-squared: 0.91


의사결정 트리 모델:
RMSE: 16968.68
MAE